In [2]:
options(warn=-1)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,516129,27.6,1095225,58.5,834747,44.6
Vcells,990005,7.6,8388608,64.0,1606620,12.3


In [3]:
folder <- "analysis1201"

In [4]:
library(xgboost)
library(readr)
library(stringr)
library(caret)
library(car)
library(DMwR)
library(Hmisc)
library(plyr)
library(dplyr)
library(tidyr)
library(data.table)
library(DescTools)
library(Matrix)
library(glmnet)
library(plotrix)
library(Ckmeans.1d.dp)

set.seed(0)
setwd("C:/Users/Jeane/MSDS/Proj2")

Loading required package: lattice
Loading required package: ggplot2
Loading required package: carData
Loading required package: grid
Loading required package: survival

Attaching package: 'survival'

The following object is masked from 'package:caret':

    cluster

Loading required package: Formula

Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units


Attaching package: 'plyr'

The following objects are masked from 'package:Hmisc':

    is.discrete, summarize

The following object is masked from 'package:DMwR':

    join


Attaching package: 'dplyr'

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:Hmisc':

    src, summarize

The following object is masked from 'package:car':

    recode

The following object is masked from 'package:xgboost':

    slice

The following objects are masked from 'pack

In [5]:
'%nin%' <- Negate('%in%')
'%in_v%' <- function(x, y) x[x %in% y] 
'%nin_v%' <- function(x, y) x[!x %in% y] 
'%in_d%' <- function(x, y) x[names(x) %in% y] 
'%nin_d%' <- function(x, y) x[!names(x) %in% y] 
'%+%' <- function(x, y) paste0(x, y)

In [6]:
labels <- readRDS("cache/labels.RData")
#sample_submission_NDF <- readRDS("cache/sample_submission_NDF.RData")
#df_all <- readRDS("cache/df_all.RData")
df_all_feats <- readRDS(("cache/df_all_feats.RData"))
age_x_stack<- readRDS(paste0("cache/",folder,"/age_stack.RData"))
age_x_test_stack<- readRDS(paste0("cache/",folder,"/age_test_stack.RData"))


In [7]:
X_all <- subset(df_all_feats, is.na(value)==F)

In [8]:
X_all <- bind_rows(X_all,
                   age_x_stack,
                   age_x_test_stack)                 
   

In [9]:
X_all <- subset(X_all, id != "")

In [10]:
X_all$feature_name <- X_all$feature

In [11]:
X_all$feature <- as.numeric(as.factor(X_all$feature))

X_all_feature <- X_all[!duplicated(X_all$feature), c("feature", "feature_name")]
X_all_feature <- X_all_feature[order(X_all_feature$feature),]

X_all$id_num <- as.numeric(as.factor(X_all$id))

In [12]:
X_all_id <- X_all %>% distinct(id_num, .keep_all=TRUE)
str(X_all_id)

Classes 'tbl_df', 'tbl' and 'data.frame':	275547 obs. of  5 variables:
 $ id          : chr  "gxn3p5htnn" "820tgsjxq7" "4ft3gnwmtx" "bjjt8pjhuk" ...
 $ feature     : num  266 266 266 266 266 266 266 266 266 266 ...
 $ value       : num  2009 2009 2009 2009 2009 ...
 $ feature_name: chr  "tfa_year" "tfa_year" "tfa_year" "tfa_year" ...
 $ id_num      : num  129856 61844 34121 88356 63040 ...


In [13]:
X_all_id <- data.frame(id = X_all_id$id, id_num = X_all_id$id_num)
X_all_id <- dplyr::left_join(X_all_id, labels, by = "id", copy = TRUE)

In [14]:
str(X_all_id)

'data.frame':	275547 obs. of  3 variables:
 $ id                 : chr  "gxn3p5htnn" "820tgsjxq7" "4ft3gnwmtx" "bjjt8pjhuk" ...
 $ id_num             : num  129856 61844 34121 88356 63040 ...
 $ country_destination: chr  "NDF" "NDF" "US" "other" ...


In [15]:
X_all_id <- X_all_id %>%
  dplyr::mutate(
    country_destination_num =recode(country_destination,'NDF'=0, 'US'=1, 
                                    'other'=2, 'FR'=3, 'CA'=4, 'GB'=5, 
                                    'ES'=6, 'IT'=7, 'PT'=8, 'NL'=9,
                                    'DE'=10, 'AU'=11))

X_all_id <- X_all_id %>%
  dplyr::arrange(id_num)
y_all <- X_all_id$country_destination_num

In [16]:
head(X_all_id)
str(y_all)

id,id_num,country_destination,country_destination_num
00023iyk9l,1,US,1
0005ytdols,2,NDF,0
000guo2307,3,US,1
000wc9mlv3,4,US,1
0010k6l0om,5,NA,NA
0012yo8hu2,6,other,2


 num [1:275547] 1 0 1 1 NA 2 1 0 0 1 ...


In [17]:

X_all <- na.omit(X_all)
X_all_sp <- sparseMatrix(i = X_all$id_num,
                         j = X_all$feature,
                         x = X_all$value)
dim(X_all_sp)


[1] 275547    269

In [18]:
unique(X_all$feature_name)

[1] "tfa_year"                                            
  [2] "tfa_month"                                           
  [3] "tfa_yearmonth"                                       
  [4] "tfa_yearmonthday"                                    
  [5] "tfa_yearmonthweek"                                   
  [6] "tfa_day"                                             
  [7] "tfa_week"                                            
  [8] "dac_lag"                                             
  [9] "dfb_dac_lag"                                         
 [10] "dfb_tfa_lag"                                         
 [11] "age_cln2"                                            
 [12] "dac_year"                                            
 [13] "dac_month"                                           
 [14] "dac_yearmonth"                                       
 [15] "dac_yearmonthday"                                    
 [16] "dac_yearmonthweek"                                   
 [17] "dac_day"                                             
 [18] "dac_week"                                            
 [19] "gender_-unknown-"                                    
 [20] "gender_MALE"                                         
 [21] "gender_FEMALE"                                       
 [22] "gender_OTHER"                                        
 [23] "first_affiliate_tracked_untracked"                   
 [24] "first_affiliate_tracked_omg"                         
 [25] "first_affiliate_tracked_NA"                          
 [26] "first_affiliate_tracked_linked"                      
 [27] "first_affiliate_tracked_tracked-other"               
 [28] "first_affiliate_tracked_product"                     
 [29] "first_affiliate_tracked_marketing"                   
 [30] "first_affiliate_tracked_local ops"                   
 [31] "signup_app_Web"                                      
 [32] "signup_app_Moweb"                                    
 [33] "signup_app_iOS"                                      
 [34] "signup_app_Android"                                  
 [35] "first_device_type_Mac Desktop"                       
 [36] "first_device_type_Windows Desktop"                   
 [37] "first_device_type_iPhone"                            
 [38] "first_device_type_Other/Unknown"                     
 [39] "first_device_type_Desktop (Other)"                   
 [40] "first_device_type_Android Tablet"                    
 [41] "first_device_type_iPad"                              
 [42] "first_device_type_Android Phone"                     
 [43] "first_device_type_SmartPhone (Other)"                
 [44] "first_browser_Chrome"                                
 [45] "first_browser_IE"                                    
 [46] "first_browser_Firefox"                               
 [47] "first_browser_Safari"                                
 [48] "first_browser_-unknown-"                             
 [49] "first_browser_Mobile Safari"                         
 [50] "first_browser_Chrome Mobile"                         
 [51] "first_browser_RockMelt"                              
 [52] "first_browser_Chromium"                              
 [53] "first_browser_Android Browser"                       
 [54] "first_browser_AOL Explorer"                          
 [55] "first_browser_Palm Pre web browser"                  
 [56] "first_browser_Mobile Firefox"                        
 [57] "first_browser_Opera"                                 
 [58] "first_browser_TenFourFox"                            
 [59] "first_browser_IE Mobile"                             
 [60] "first_browser_Apple Mail"                            
 [61] "first_browser_Silk"                                  
 [62] "first_browser_Camino"                                
 [63] "first_browser_Arora"                                 
 [64] "first_browser_BlackBerry Browser"                    
 [65] "first_browser_SeaMonkey"                             
 [66] "first_browser_Iron"           

In [19]:
X_all_dense <- data.frame(as.matrix(X_all_sp))
names(X_all_dense) <- X_all_feature$feature_name

X_dense <- subset(X_all_dense, dac_yearmonth %nin% c("201407", "201408", "201409"))
X_index <- as.integer(row.names(X_all_dense[X_all_dense$dac_yearmonth %nin% c("201407", "201408", "201409"), ]))
dim(X_dense)
X_train_dense <- subset(X_dense, dac_yearmonthweek %nin% c("20140521", "20140522", "20140622", "20140623", "20140624", "20140625", "20140626"))
X_train_index <- as.integer(row.names(X_dense[X_dense$dac_yearmonthweek %nin% c("20140521", "20140522", "20140622", "20140623", "20140624", "20140625", "20140626"), ]))
dim(X_train_dense)

[1] 213451    269

[1] 193116    269

In [22]:
X_test_dense <- subset(X_all_dense, dac_yearmonth %in% c("201407", "201408", "201409"))
X_test_index <- as.integer(row.names(X_all_dense[X_all_dense$dac_yearmonth %in% c("201407", "201408", "201409"), ]))
dim(X_test_dense)

[1] 62096   269

In [23]:
X_valid_dense <- subset(X_dense, dac_yearmonthweek %in% c("20140521", "20140522", "20140622", "20140623", "20140624", "20140625", "20140626"))
X_valid_index <- as.integer(row.names(X_dense[X_dense$dac_yearmonthweek %in% c("20140521", "20140522", "20140622", "20140623", "20140624", "20140625", "20140626"), ]))
dim(X_valid_dense)

[1] 20335   269

In [26]:
dim(X_all_dense)
print(dim(X_train_dense[0])+dim(X_test_dense[0])+dim(X_valid_dense[0]))

[1] 275547    269

[1] 275547      0


In [27]:
y <- y_all[X_index]
y_train <- y_all[X_train_index]
y_valid <- y_all[X_valid_index]
y_test <- y_all[X_test_index]

X_id <- X_all_id[X_index,]
X_train_id <- X_all_id[X_train_index,]
X_valid_id <- X_all_id[X_valid_index,]
X_test_id <- X_all_id[X_test_index,]

X_sp <- data.matrix(X_dense)
X_train_sp <- data.matrix(X_train_dense)
X_valid_sp <- data.matrix(X_valid_dense)
X_test_sp <- data.matrix(X_test_dense)

dX <- xgb.DMatrix(X_sp, label = y, missing = -99999)
dX_test <- xgb.DMatrix(X_test_sp, label = y_test, missing = -99999)

dX_train <- xgb.DMatrix(X_train_sp, label = y_train, missing = -99999)
dX_valid <- xgb.DMatrix(X_valid_sp, label = y_valid, missing = -99999)

saveRDS(X_all_feature, paste0("cache/",folder,"/X_all_feature.RData"))
saveRDS(X_id, paste0("cache/",folder,"/X_id.RData"))
saveRDS(X_test_id, paste0("cache/",folder,"/X_test_id.RData"))
saveRDS(X_train_id, paste0("cache/",folder,"/X_train_id.RData"))
saveRDS(X_valid_id, paste0("cache/",folder,"/X_valid_id.RData"))

saveRDS(y, paste0("cache/",folder,"/y.RData"))
saveRDS(y_test, paste0("cache/",folder,"/y_test.RData"))
saveRDS(y_train, paste0("cache/",folder,"/y_train.RData"))
saveRDS(y_valid, paste0("cache/",folder,"/y_valid.RData"))

saveRDS(X_sp, paste0("cache/",folder,"/X_sp.RData"))
saveRDS(X_test_sp, paste0("cache/",folder,"/X_test_sp.RData"))
saveRDS(X_train_sp, paste0("cache/",folder,"/X_train_sp.RData"))
saveRDS(X_valid_sp, paste0("cache/",folder,"/X_valid_sp.RData"))

xgb.DMatrix.save(dX, paste0("cache/",folder,"/dX.xgb.DMatrix.data"))
xgb.DMatrix.save(dX_test, paste0("cache/",folder,"/dX_test.xgb.DMatrix.data"))
xgb.DMatrix.save(dX_train, paste0("cache/",folder,"/dX_train.xgb.DMatrix.data"))
xgb.DMatrix.save(dX_valid, paste0("cache/",folder,"/dX_valid.xgb.DMatrix.data"))


[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

In [ ]:
X_all_feature <- readRDS(paste0("cache/",folder,"/X_all_feature.RData"))
X_id <- readRDS(paste0("cache/",folder,"/X_id.RData"))
X_test_id <- readRDS(paste0("cache/",folder,"/X_test_id.RData"))
X_train_id <- readRDS(paste0("cache/",folder,"/X_train_id.RData"))
X_valid_id <- readRDS(paste0("cache/",folder,"/X_valid_id.RData"))

y <- readRDS(paste0("cache/",folder,"/y.RData"))
y_test <- readRDS(paste0("cache/",folder,"/y_test.RData"))
y_train <- readRDS(paste0("cache/",folder,"/y_train.RData"))
y_valid <- readRDS(paste0("cache/",folder,"/y_valid.RData"))

X_sp <- readRDS(paste0("cache/",folder,"/X_sp.RData"))
X_test_sp <- readRDS(paste0("cache/",folder,"/X_test_sp.RData"))
X_train_sp <- readRDS(paste0("cache/",folder,"/X_train_sp.RData"))
X_valid_sp <- readRDS(paste0("cache/",folder,"/X_valid_sp.RData"))

dX <- xgb.DMatrix(paste0("cache/",folder,"/dX.xgb.DMatrix.data"))
dX_test <- xgb.DMatrix(paste0("cache/",folder,"/dX_test.xgb.DMatrix.data"))
dX_train <- xgb.DMatrix(paste0("cache/",folder,"/dX_train.xgb.DMatrix.data"))
dX_test <- xgb.DMatrix(paste0("cache/",folder,"/dX_test.xgb.DMatrix.data"))
